In [3]:
from binance import Client
from datetime import datetime, timedelta
client = Client(api_key="jM9zEeTTzT7R7QGlNrh5tUpOc8l71hOxu7CewCL4M0NvbBp3ilgmEwMOOjJfCRVB",api_secret="iZxFmlz5bvROsHQPZTP5hlJtsxYjOu8wocOlLWzIDdhMzEQu7KyyHB2DseOYQ66s",testnet=True)

import os
from google.cloud.sql.connector import Connector
from sqlalchemy import create_engine, text
import pg8000.native
import pandas as pd
from time import sleep

os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = 'C:/Users/tinma/OneDrive/Escritorio/application_default_credentials.json'

connector = Connector()

def getconn() -> pg8000.native.Connection:
    conn: pg8000.native.Connection = connector.connect(
        "semiotic-pact-388619:us-central1:tradingbot",
        "pg8000",
        user="postgres",
        password="GuruSat.3",
        db="criptos"
    )
    return conn

connection = create_engine('postgresql+pg8000://',creator=getconn)

def MACD(df):
    df['EMA12'] = df.Precio.ewm(span=12).mean()
    df['EMA26'] = df.Precio.ewm(span=26).mean()
    df['MACD'] = df.EMA12 - df.EMA26
    df['Signal'] = df.MACD.ewm(span=9).mean()
    df['Histogram'] = df.MACD - df.Signal
    # print('Indicadores agregados')

def señal_MACD(df):
    buy, sell = [],[]
    for i in range(7,len(df)):
        if df.Histogram.iloc[i] > 0 and df.Histogram.iloc[i-1] < 0:
            buy.append(i)
        elif df.Histogram.iloc[i] < 0 and df.Histogram.iloc[i-1] > 0:
            sell.append(i)
    return buy, sell

In [4]:
client.get_account()

{'makerCommission': 0,
 'takerCommission': 0,
 'buyerCommission': 0,
 'sellerCommission': 0,
 'commissionRates': {'maker': '0.00000000',
  'taker': '0.00000000',
  'buyer': '0.00000000',
  'seller': '0.00000000'},
 'canTrade': True,
 'canWithdraw': False,
 'canDeposit': False,
 'brokered': False,
 'requireSelfTradePrevention': False,
 'preventSor': False,
 'updateTime': 1688617091784,
 'accountType': 'SPOT',
 'balances': [{'asset': 'BNB',
   'free': '1000.00000000',
   'locked': '0.00000000'},
  {'asset': 'BTC', 'free': '1.00500000', 'locked': '0.00000000'},
  {'asset': 'BUSD', 'free': '10000.00000000', 'locked': '0.00000000'},
  {'asset': 'ETH', 'free': '100.20000000', 'locked': '0.00000000'},
  {'asset': 'LTC', 'free': '500.00000000', 'locked': '0.00000000'},
  {'asset': 'TRX', 'free': '500000.00000000', 'locked': '0.00000000'},
  {'asset': 'USDT', 'free': '9467.49616930', 'locked': '0.00000000'},
  {'asset': 'XRP', 'free': '50000.00000000', 'locked': '0.00000000'}],
 'permissions': 

In [5]:
path = 'C:/Users/tinma/OneDrive/Escritorio/tradingbot/trade_logs/testnet/logs/Trades/MACD/streaming/'
balances_p = []
PAIR = 'BTCUSDT'
asset = 'btc'
LOCALIDAD_SEÑAL_ENTRADA = 10 # SEGUNDOS
LOCALIDAD_SEÑAL_SALIDA = 10 # SEGUNDOS
UMBRAL_PERDIDA = - 5 # DÓLARES
open_order = False
balance_trade_anterior = 0
balance_trade_p = 0
open_order = False
tiempo_inicial = datetime.utcnow()
estrategia = 'MACD'
cantidad_trade = 0.005

while True:

######################## STREAMING Y ESTRATEGIA.

    day = datetime.utcnow() - timedelta(seconds=1000)

    with connection.connect() as conn:
        df =conn.execute(text(""" SELECT * FROM "{PAIR}" WHERE "Tiempo" > '{day}' """.format(day=day,PAIR=PAIR))).mappings().all()
    df = pd.DataFrame(df)[['Tiempo', 'Precio']].set_index('Tiempo')
    df = df.sort_values(by='Tiempo')
    MACD(df)
    buy, sell = señal_MACD(df)

    buyprices = df.Precio.iloc[buy]
    sellprices = df.Precio.iloc[sell]

######################## MÉTODO DE ENTRADA POR MEDIO DE SEÑAL.

    if open_order == False:

        DIFF_SEÑAL_ENTRADA = (datetime.utcnow() - buyprices.index[-1]).total_seconds()

        if DIFF_SEÑAL_ENTRADA < LOCALIDAD_SEÑAL_ENTRADA:
            open_order = True
            buy_order = client.create_order(
                        symbol=PAIR,
                        side=Client.SIDE_BUY,
                        type=Client.ORDER_TYPE_MARKET,
                        quantity=cantidad_trade,
                        recvWindow=60000
                    )
            tiempo_entrada = pd.to_datetime(buy_order['transactTime'], unit='ms')
            precio_entrada = float(buy_order['fills'][0]['price'])
            print(f'Se realizó una compra al precio: {precio_entrada}, en el momento: {tiempo_entrada}')

    if open_order == True:

######################## MÉTODO DE SALIDA POR MEDIO DEL UMBRAL DE PÉRIDA.

        if df.Precio[-1] - precio_entrada < UMBRAL_PERDIDA:
            open_order = False
            sell_order = client.create_order(
                        symbol=PAIR,
                        side=Client.SIDE_SELL,
                        type=Client.ORDER_TYPE_MARKET,
                        quantity=cantidad_trade,
                        recvWindow=60000
                    )
            Metodo = 'UMBRAL_PERDIDA'
            tiempo_salida = pd.to_datetime(sell_order['transactTime'], unit='ms')
            precio_salida = float(sell_order['fills'][0]['price'])
            balance_trade_p = (precio_salida - precio_entrada)*cantidad_trade
            balances_p.append(balance_trade_p)
            
######################## MÉTODO DE SALIDA POR MEDIO DE SEÑAL

    if open_order == True:

        DIFF_SEÑAL_SALIDA = (datetime.utcnow() - sellprices.index[-1]).total_seconds()

        if DIFF_SEÑAL_SALIDA < LOCALIDAD_SEÑAL_SALIDA and sellprices.index[-1] > buyprices.index[-1]:
            open_order = False
            sell_order = client.create_order(
                        symbol=PAIR,
                        side=Client.SIDE_SELL,
                        type=Client.ORDER_TYPE_MARKET,
                        quantity=cantidad_trade,
                        recvWindow=60000
                    )
            Metodo = 'SEÑAL'
            tiempo_salida = pd.to_datetime(sell_order['transactTime'], unit='ms')
            precio_salida = float(sell_order['fills'][0]['price'])
            balance_trade_p = (precio_salida - precio_entrada)*cantidad_trade
            balances_p.append(balance_trade_p)
                    
######################## LOGS

    if balance_trade_anterior != balance_trade_p:
        balance_trade_anterior = balance_trade_p
        if balance_trade_p>0:
            print(f'Se realizó una venta al precio: {precio_salida}, en el momento: {tiempo_salida}. Ganancia de: {balance_trade_p} dólares. \nMétodo: {Metodo}')
            trade_log = pd.DataFrame([[precio_entrada, tiempo_entrada, precio_salida, tiempo_salida, (balance_trade_p)]],
                                columns=['Precio_Entrada','Tiempo_Entrada','Precio_Salida','Tiempo_Salida','Ganancia'])
            if os.path.exists(path+f'trades_ganados_{asset}.csv'):
                trade_log.to_csv(path+f'trades_ganados_{asset}.csv',
                                mode='a',
                                index=False,
                                header=False
                                )
            else:
                trade_log.to_csv(path+f'trades_ganados_{asset}.csv',
                                index=False,
                                header=True,
                                )
        elif balance_trade_p==0:
            print('No hubo pérdidas ni ganancias del trade')
        else:
            print(f'Se realizó una venta al precio: {precio_salida}, en el momento: {tiempo_salida}. Pérdida de: {balance_trade_p} dólares. \nMétodo: {Metodo}')
            trade_log = pd.DataFrame([[precio_entrada, tiempo_entrada, precio_salida, tiempo_salida, (balance_trade_p),Metodo]],
                                columns=['Precio_Entrada','Tiempo_Entrada','Precio_Salida','Tiempo_Salida','Perdida','Metodo'])
            if os.path.exists(path+f'trades_perdidos_{asset}.csv'):
                trade_log.to_csv(path+f'trades_perdidos_{asset}.csv',
                                mode='a',
                                index=False,
                                header=False
                                )
            else:
                trade_log.to_csv(path+f'trades_perdidos_{asset}.csv',
                                index=False,
                                header=True,
                                )

Se realizó una compra al precio: 30358.89, en el momento: 2023-07-05 13:56:08.490380
Se realizó una venta al precio: 30352.79, en el momento: 2023-07-05 13:56:30.715878. Pérdida de: -0.030499999999992724 dólares. 
Método: UMBRAL_PERDIDA
Se realizó una compra al precio: 30360.99, en el momento: 2023-07-05 13:56:43.829640
Se realizó una venta al precio: 30362.19, en el momento: 2023-07-05 13:57:13.248152. Ganancia de: 0.005999999999985448 dólares. 
Método: SEÑAL
Se realizó una compra al precio: 30361.26, en el momento: 2023-07-05 13:57:34.996051
Se realizó una venta al precio: 30357.3, en el momento: 2023-07-05 13:57:52.469639. Pérdida de: -0.019799999999995634 dólares. 
Método: SEÑAL
Se realizó una compra al precio: 30357.31, en el momento: 2023-07-05 13:58:22.017960
Se realizó una venta al precio: 30357.3, en el momento: 2023-07-05 13:59:53.240159. Pérdida de: -5.000000001018634e-05 dólares. 
Método: SEÑAL
Se realizó una compra al precio: 30357.69, en el momento: 2023-07-05 14:00:31.90

KeyboardInterrupt: 

In [5]:
path = 'C:/Users/tinma/OneDrive/Escritorio/tradingbot/trade_logs/testnet/logs/Sesiones/MACD_MACD+logicv1/'

# sleep(5)

if open_order:
    sell_order = client.create_order(
                            symbol=PAIR,
                            side=Client.SIDE_SELL,
                            type=Client.ORDER_TYPE_MARKET,
                            quantity=cantidad_trade
                        )
    open_order = False
    tiempo_salida = pd.to_datetime(sell_order['transactTime'], unit='ms')
    precio_salida = float(sell_order['fills'][0]['price'])
    balance_trade_p = (precio_salida - precio_entrada)*cantidad_trade
    balances_p.append(balance_trade_p)

if tiempo_entrada > tiempo_salida:
    tiempo_final = tiempo_entrada
elif tiempo_entrada < tiempo_salida:
    tiempo_final = tiempo_salida

if sum(balances_p) > 0:
    session_log = pd.DataFrame([[len(balances_p), sum(balances_p), sum(balances_p)/len(balances_p), PAIR, estrategia, tiempo_inicial, tiempo_final,cantidad_trade]],
             columns=['Cantidad_Trades','Ganancia','Ganancia_Promedio_xTrade','PAR','Estrategia','Tiempo_inicial','Tiempo_final','Cantidad_trade'])
    if os.path.exists(path+'Sesiones_Ganadas.csv'):
        session_log.to_csv(path+'Sesiones_Ganadas.csv',
                        mode='a',
                        index=False,
                        header=False
                        )
    else:
        session_log.to_csv(path+'Sesiones_Ganadas.csv',
                        index=False,
                        header=True
                        )
elif sum(balances_p) < 0:
    session_log = pd.DataFrame([[len(balances_p), sum(balances_p), sum(balances_p)/len(balances_p), PAIR, estrategia, tiempo_inicial, tiempo_final,cantidad_trade]],
             columns=['Cantidad_Trades','Perdida','Ganancia_Promedio_xTrade','PAR','Estrategia','Tiempo_inicial','Tiempo_final','Cantidad_trade'])
    if os.path.exists(path+'Sesiones_Perdidas.csv'):
        session_log.to_csv(path+'Sesiones_Perdidas.csv',
                        mode='a',
                        index=False,
                        header=False
                        )
    else:
        session_log.to_csv(path+'Sesiones_Perdidas.csv',
                        index=False,
                        header=True
                        )

NameError: name 'open_order' is not defined

In [7]:
session_log

,Cantidad_Trades,Ganancia,Ganancia_Promedio_xTrade,PAR,Estrategia,Tiempo_inicial,Tiempo_final,Cantidad_trade
0,40,0.4166,0.010415,BTCUSDT,MACD,2023-07-05 13:55:30.197377,2023-07-05 14:33:52.602318,0.005


In [120]:
client.get_account()

{'makerCommission': 0,
 'takerCommission': 0,
 'buyerCommission': 0,
 'sellerCommission': 0,
 'commissionRates': {'maker': '0.00000000',
  'taker': '0.00000000',
  'buyer': '0.00000000',
  'seller': '0.00000000'},
 'canTrade': True,
 'canWithdraw': False,
 'canDeposit': False,
 'brokered': False,
 'requireSelfTradePrevention': False,
 'updateTime': 1688510192529,
 'accountType': 'SPOT',
 'balances': [{'asset': 'BNB',
   'free': '1000.00000000',
   'locked': '0.00000000'},
  {'asset': 'BTC', 'free': '1.00000000', 'locked': '0.00000000'},
  {'asset': 'BUSD', 'free': '10000.00000000', 'locked': '0.00000000'},
  {'asset': 'ETH', 'free': '100.00000000', 'locked': '0.00000000'},
  {'asset': 'LTC', 'free': '500.00000000', 'locked': '0.00000000'},
  {'asset': 'TRX', 'free': '500000.00000000', 'locked': '0.00000000'},
  {'asset': 'USDT', 'free': '9949.69067169', 'locked': '0.00000000'},
  {'asset': 'XRP', 'free': '50000.00000000', 'locked': '0.00000000'}],
 'permissions': ['SPOT']}

In [111]:
len(balances_p)

109

In [113]:
sum(balances_p) + 2.6322000000000116

0.9602499999999239

In [42]:
balances = pd.DataFrame(balances_p,columns=['balance'])
balances

,balance
0,-0.00855
1,-0.00865
2,0.00120
3,-0.00850
4,-0.00545
...,...
104,-0.00840
105,-0.01080
106,0.06650
107,-0.00045


In [93]:
client.get_all_orders(symbol='BTCUSDT',limit=200)

[{'symbol': 'BTCUSDT',
  'orderId': 11884068,
  'orderListId': -1,
  'clientOrderId': 'xqAnpWSGWLbKkYgzBC0hjl',
  'price': '0.00000000',
  'origQty': '0.00500000',
  'executedQty': '0.00500000',
  'cummulativeQuoteQty': '155.29140000',
  'status': 'FILLED',
  'timeInForce': 'GTC',
  'type': 'MARKET',
  'side': 'SELL',
  'stopPrice': '0.00000000',
  'icebergQty': '0.00000000',
  'time': 1688473575706,
  'updateTime': 1688473575706,
  'isWorking': True,
  'workingTime': 1688473575706,
  'origQuoteOrderQty': '0.00000000',
  'selfTradePreventionMode': 'NONE'},
 {'symbol': 'BTCUSDT',
  'orderId': 11884178,
  'orderListId': -1,
  'clientOrderId': 'qHRFEhVcZRHffPOpXP5lPB',
  'price': '0.00000000',
  'origQty': '0.00500000',
  'executedQty': '0.00500000',
  'cummulativeQuoteQty': '155.28450000',
  'status': 'FILLED',
  'timeInForce': 'GTC',
  'type': 'MARKET',
  'side': 'SELL',
  'stopPrice': '0.00000000',
  'icebergQty': '0.00000000',
  'time': 1688473602093,
  'updateTime': 1688473602093,
  

In [109]:
dfx = pd.DataFrame(client.get_my_trades(symbol='BTCUSDT',limit=400))
dfx

,symbol,id,orderId,orderListId,price,qty,quoteQty,commission,commissionAsset,time,isBuyer,isMaker,isBestMatch
0,BTCUSDT,3341741,11855809,-1,31022.99000000,0.00500000,155.11495000,0.00000000,BTC,1688468357115,True,False,True
1,BTCUSDT,3341742,11855810,-1,31022.99000000,0.00500000,155.11495000,0.00000000,BTC,1688468357318,True,False,True
2,BTCUSDT,3341880,11856198,-1,31024.31000000,0.00500000,155.12155000,0.00000000,USDT,1688468431944,False,False,True
3,BTCUSDT,3341954,11856501,-1,31022.53000000,0.00500000,155.11265000,0.00000000,BTC,1688468492658,True,False,True
4,BTCUSDT,3342196,11857376,-1,31018.18000000,0.00500000,155.09090000,0.00000000,USDT,1688468664204,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...
395,BTCUSDT,3360753,11911719,-1,31006.37000000,0.00500000,155.03185000,0.00000000,USDT,1688478956194,False,False,True
396,BTCUSDT,3360921,11911945,-1,31005.63000000,0.00500000,155.02815000,0.00000000,BTC,1688478986759,True,False,True
397,BTCUSDT,3360922,11911946,-1,31005.63000000,0.00500000,155.02815000,0.00000000,BTC,1688478986824,True,False,True
398,BTCUSDT,3360980,11912194,-1,31003.18000000,0.00500000,155.01590000,0.00000000,USDT,1688479030542,False,False,True


In [101]:
dfx['price'] = dfx['price'].astype(float)

In [110]:
dfx[0:50]

,symbol,id,orderId,orderListId,price,qty,quoteQty,commission,commissionAsset,time,isBuyer,isMaker,isBestMatch
0,BTCUSDT,3341741,11855809,-1,31022.99000000,0.00500000,155.11495000,0.00000000,BTC,1688468357115,True,False,True
1,BTCUSDT,3341742,11855810,-1,31022.99000000,0.00500000,155.11495000,0.00000000,BTC,1688468357318,True,False,True
2,BTCUSDT,3341880,11856198,-1,31024.31000000,0.00500000,155.12155000,0.00000000,USDT,1688468431944,False,False,True
3,BTCUSDT,3341954,11856501,-1,31022.53000000,0.00500000,155.11265000,0.00000000,BTC,1688468492658,True,False,True
4,BTCUSDT,3342196,11857376,-1,31018.18000000,0.00500000,155.09090000,0.00000000,USDT,1688468664204,False,False,True
5,BTCUSDT,3342225,11857437,-1,31014.45000000,0.00500000,155.07225000,0.00000000,USDT,1688468672891,False,False,True
6,BTCUSDT,3342234,11857548,-1,31014.46000000,0.00500000,155.07230000,0.00000000,BTC,1688468699117,True,False,True
7,BTCUSDT,3342235,11857549,-1,31014.46000000,0.00500000,155.07230000,0.00000000,BTC,1688468699181,True,False,True
8,BTCUSDT,3342315,11857710,-1,31011.89000000,0.00125700,38.98194573,0.00000000,USDT,1688468725229,False,False,True
9,BTCUSDT,3342316,11857710,-1,31011.61000000,0.00374300,116.07645623,0.00000000,USDT,1688468725229,False,False,True


In [87]:
for i in range(100,150):
    if client.get_my_trades(symbol='BTCUSDT',limit=250)[-i]['price'] == '31038.31' and client.get_my_trades(symbol='BTCUSDT',limit=250)[-i]['isBuyer'] == True:
        print(client.get_my_trades(symbol='BTCUSDT')[-i])

In [90]:
client.get_my_trades(symbol='BTCUSDT')[-172]

{'symbol': 'BTCUSDT',
 'id': 3353156,
 'orderId': 11887812,
 'orderListId': -1,
 'price': '31043.30000000',
 'qty': '0.00161300',
 'quoteQty': '50.07284290',
 'commission': '0.00000000',
 'commissionAsset': 'USDT',
 'time': 1688474320607,
 'isBuyer': False,
 'isMaker': False,
 'isBestMatch': True}

In [53]:
sum(balances[balances['balance'] < 0]['balance']) + 2.63220

-0.8564000000000607

In [54]:
sum(balances[balances['balance'] > 0]['balance'])

1.8166499999999723

In [13]:
buy_order = client.create_order(
                        symbol='BTCUSDT',
                        side=Client.SIDE_BUY,
                        type=Client.ORDER_TYPE_MARKET,
                        quantity=0.1
                    )

In [15]:
sell_order = client.create_order(
                        symbol='BTCUSDT',
                        side=Client.SIDE_SELL,
                        type=Client.ORDER_TYPE_MARKET,
                        quantity=0.01
                    )

In [117]:
client.get_account()

{'makerCommission': 0,
 'takerCommission': 0,
 'buyerCommission': 0,
 'sellerCommission': 0,
 'commissionRates': {'maker': '0.00000000',
  'taker': '0.00000000',
  'buyer': '0.00000000',
  'seller': '0.00000000'},
 'canTrade': True,
 'canWithdraw': False,
 'canDeposit': False,
 'brokered': False,
 'requireSelfTradePrevention': False,
 'updateTime': 1688510143385,
 'accountType': 'SPOT',
 'balances': [{'asset': 'BNB',
   'free': '1000.00000000',
   'locked': '0.00000000'},
  {'asset': 'BTC', 'free': '1.00500000', 'locked': '0.00000000'},
  {'asset': 'BUSD', 'free': '10000.00000000', 'locked': '0.00000000'},
  {'asset': 'ETH', 'free': '100.00000000', 'locked': '0.00000000'},
  {'asset': 'LTC', 'free': '500.00000000', 'locked': '0.00000000'},
  {'asset': 'TRX', 'free': '500000.00000000', 'locked': '0.00000000'},
  {'asset': 'USDT', 'free': '9795.84967169', 'locked': '0.00000000'},
  {'asset': 'XRP', 'free': '50000.00000000', 'locked': '0.00000000'}],
 'permissions': ['SPOT']}